# Use the best AutoML generated model to batch score credit worthiness

<img src="https://raw.githubusercontent.com/QuentinAmbard/databricks-demo/main/retail/resources/images/lakehouse-retail/lakehouse-retail-churn-ml-experiment.png" style="float: right" width="600px">


Databricks AutoML runs experiments across a grid and creates many models and metrics to determine the best models among all trials. This is a glass-box approach to create a baseline model, meaning we have all the code artifacts and experiments available afterwards. 

Here, we selected the Notebook from the best run from the AutoML experiment.

All the code below has been automatically generated. As data scientists, we can tune it based on our business knowledge, or use the generated model as-is.

This saves data scientists hours of developement and allows team to quickly bootstrap and validate new projects, especally when we may not know the predictors for alternative data such as the telco payment data.

<!-- Collect usage data (view). Remove it to disable collection. View README for more details.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=lakehouse&org_id=1444828305810485&notebook=%2F03-Data-Science-ML%2F03.3-Batch-Scoring-credit-decisioning&demo_name=lakehouse-fsi-credit&event=VIEW&path=%2F_dbdemos%2Flakehouse%2Flakehouse-fsi-credit%2F03-Data-Science-ML%2F03.3-Batch-Scoring-credit-decisioning&version=1&user_hash=7804490f0d3be4559d29a7b52959f461489c4ee5e35d4afc7b55f311360ac589">

### A cluster has been created for this demo
To run this demo, just select the cluster `dbdemos-lakehouse-fsi-credit-junyi_tiong` from the dropdown menu ([open cluster configuration](https://e2-demo-field-eng.cloud.databricks.com/#setting/clusters/0922-083237-e7fg83pu/configuration)). <br />
*Note: If the cluster was deleted after 30 days, you can re-create it with `dbdemos.create_cluster('lakehouse-fsi-credit')` or re-install the demo: `dbdemos.install('lakehouse-fsi-credit')`*

In [0]:
%pip install mlflow==2.19.0
dbutils.library.restartPython()

In [0]:
%run ../_resources/00-setup $reset_all_data=false


## Running batch inference to score our existing database

<img src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/main/images/fsi/credit_decisioning/fsi-credit-decisioning-ml-5.png" style="float: right" width="800px">

<br/><br/>
Now that our model was created and deployed in production within the MLFlow registry.

<br/>
We can now easily load it calling the `Production` stage, and use it in any Data Engineering pipeline (a job running every night, in streaming or even within a Delta Live Table pipeline).

<br/>

We'll then save this information as a new table without our FS database, and start building dashboards and alerts on top of it to run live analysis.

In [0]:
model_name = "dbdemos_fsi_credit_decisioning"
import mlflow
mlflow.set_registry_uri('databricks-uc')

# Load model as a Spark UDF.
loaded_model = mlflow.pyfunc.spark_udf(spark, model_uri=f"models:/{catalog}.{db}.{model_name}@prod", result_type='double')

In [0]:
features = loaded_model.metadata.get_input_schema().input_names()

underbanked_df = spark.table("credit_decisioning_features").fillna(0) \
                   .withColumn("prediction", loaded_model(F.struct(*features))).cache()

display(underbanked_df)


In the scored data frame above, we have essentially created an end-to-end process to predict credit worthiness for any customer, regardless of whether the customer has an existing bank account. We have a binary prediction which captures this and incorporates all the intellience from Databricks AutoML and curated features from our feature store.

In [0]:
underbanked_df.write.mode("overwrite").saveAsTable(f"underbanked_prediction")


### Next steps

* Deploy your model for real time inference with [03.4-model-serving-BNPL-credit-decisioning]($./03.4-model-serving-BNPL-credit-decisioning) to enable ```Buy Now, Pay Later``` capabilities within the bank.

Or

* Making sure your model is fair towards customers of any demographics are extremely important parts of building production-ready ML models for FSI use cases. <br/>
Explore your model with [03.5-Explainability-and-Fairness-credit-decisioning]($./03.5-Explainability-and-Fairness-credit-decisioning) on the Lakehouse.